### Load the Coordinates and Elevation CSV files

In [30]:
import pandas as pd

# Load the coordinate csv file

coordinates = pd.read_csv("clean-coordinates-data.csv")

coordinates.head()

# check the structure of the data using the info() method

,country,coordinates,country-code,continent
0,Belarus,"53.5318804783, 28.033566395",BLR,Europe
1,Germany,"51.1063634863, 10.3814938434",DEU,Europe
2,Denmark,"62.0740272545, -6.87055477617",DN1,Europe
3,Portugal,"39.5871468639, -8.59083758901",PRT,Europe
4,Andorra,"42.541326525, 1.56075624107",AND,Europe


In [32]:
# Load the elevation csv file

elevation = pd.read_csv("clean-elevation-data.csv")

elevation.head()


,country,Numeric-Elevation
0,Afghanistan,1884.0
1,Albania,708.0
2,Algeria,800.0
3,Andorra,1996.0
4,Angola,1112.0


In [38]:
# Check if all country names columns in both dataframes are the same

# Check if elements coordinates['country'] are in elevation['country']
coordinates["in_elevation"] = coordinates["country"].isin(elevation["country"])

# Inspect the result
coordinates[coordinates["in_elevation"] == False]

,country,coordinates,country-code,continent,in_elevation
10,San Marino,"43.9364661946, 12.4422225588",SMR,Europe,False
15,Monaco,"43.7398461874, 7.39897432483",MCO,Europe,False
17,Czechia,"49.7324482055, 15.3141031033",CZE,Europe,False
18,Liechtenstein,"47.1382064295, 9.54090028864",LIE,Europe,False
24,Malta,"35.921026987, 14.4037540314",MLT,Europe,False
39,Republic of Serbia,"44.2213759212, 20.790402471",SRB,Europe,False
42,Vatican,"41.9033124692, 12.4534177294",VAT,Europe,False
44,Macedonia,"41.5950049267, 21.6838304101",MKD,Europe,False



For Microstates, we will impute the average elevation of the country they are within:
- San Marino is a microstate within Italy. 
- Monaco is a microstate within France.
- Vatican City is a microstate within Italy.
- Liechtenstein is a microstate within Switzerland.

1. Rename Czechia as Czech Republic.
2. Rename Republic of Serbia as Serbia.
3. Rename Macedonia as North Macedonia.
4. Malta is an island nation in the Mediterranean Sea with Average elevation of 127 meters.


In [35]:
# Merge the two dataframes on the 'country' column

merged = pd.merge(coordinates, elevation, how="inner", on="country")

merged.__len__()


37